# word2vec и его модели

За материал этого семинара большое спасибо Лизе Кузьменко, выпускнице ФиПЛа ВШЭ.

In [1]:
import gensim

C:\Users\user\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


`gensim` -- это питоновская библиотека для семантического моделирования  

## Что можно делать с word2vec?

Можно обучать модели на своих данных. Это может пригодиться вам, если модели для вашего языка ещё нет. Или если текст документов очень специфичен (например, юридические докуенты).

А можно -- загрузить готовую модель и использовать её.

## Загрузка модели

Скачаем модель [отсюда](http://rusvectores.org/static/models/rusvectores4/RNC/ruscorpora_upos_skipgram_300_5_2018.vec.gz). Или с флешки.

(Кстати, другие модели для русского скачать можно здесь - http://rusvectores.org/ru/models/).

Модели бывают разных форматов и, в зависимости от этого, их надо по-разному считывыать. ААА

In [2]:
m_path = 'ruscorpora_upos_skipgram_300_5_2018.vec.gz'
model = gensim.models.KeyedVectors.load_word2vec_format(m_path, binary=False)

In [3]:
# делаем нормализацию
model.init_sims(replace=True)

Посмотрим, сколько в модели слов:

In [4]:
len(model.vocab)

195071

Попросим у модели 5 ближайших соседей для нашего слова и коэффициент косинусной близости для каждого:

In [11]:
model.most_similar(positive=['камень_NOUN'], topn=5)

C:\Users\user\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('каменья_NOUN', 0.7000336647033691),
 ('камешек_NOUN', 0.6860172748565674),
 ('валун_NOUN', 0.6670265197753906),
 ('глыба_NOUN', 0.6664066314697266),
 ('камушек_NOUN', 0.6610238552093506)]

Частеречные тэги нужны, поскольку это специфика скачанной модели - она была натренирована на словах, аннотированных их частями речи (и лемматизированных).

Проверим, есть ли слово в модели:

In [12]:
word1 = 'захардкодить_NOUN'
word2 = 'пирожок_NOUN'
print(word1 in model)
print(word2 in model)

False
True


Решить пропорцию!

In [39]:
print(model.most_similar(positive=['солдат_NOUN', 'университет_NOUN'], negative=['армия_NOUN'])[0][0])

студент_NOUN


C:\Users\user\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Найти лишнее!

In [44]:
print(model.doesnt_match('яблоко_NOUN груша_NOUN виноград_NOUN банан_NOUN лимон_NOUN свекла_NOUN'.split()))

свекла_NOUN


C:\Users\user\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


## Домашнее задание

### Обязательное задание всем

Напишите функцию, которая печатает 7 ближайших слов для поданного прилагательного, если оно есть в модели. А если нет -- печатает сообщение "в модели нет этого слова :("

Пример:

In [13]:
find_seven_similar('красивый_ADJ')

NameError: name 'find_seven_similar' is not defined

In [59]:
find_seven_similar('улётный_ADJ')

NameError: name 'find_seven_similar' is not defined

In [11]:
# ваш код здесь
word = input()
if word + '_ADJ' in model:
    res = model.most_similar(positive=[word + '_ADJ'], topn=7)
    for elems in res:
        print(elems[0])
else:
    print('в модели нет этого слова :(')

белый
белоснежный_ADJ
желтый_ADJ
белый_NOUN
серый_ADJ
ярко-белый_ADJ
черный_ADJ
розовый_ADJ


C:\Users\user\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


## Необязательное задание 1
(делайте только если у вас время, силы и желание, а если нет - забейте :))

Напишите функцию, 

Пример того, как определять часть речи слова с помощью pymorphy, есть [в семинаре по лемматизации](https://github.com/religofsil/mipt-summer-school/blob/master/3_lemmatization%26hw.ipynb).

In [6]:
from pymorphy2  import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()

In [7]:
word = pymorphy2_analyzer.parse(input())
print(word[0].tag.POS)

белый
ADJF


In [4]:
d = {
    'ADJF': 'ADJ',
    'ADVB': 'ADV',
    'ADJS': 'ADJ',
    'NUMR': 'NUM',
    'PREP': 'ADP',
    'CONJ': 'CCONJ',
    'INFN': 'VERB',
    'VERB': 'VERB',
    'NOUN': 'NOUN'
}

In [48]:
word = pymorphy2_analyzer.parse(input())
tag = word[0].tag.POS
word = word[0].normal_form


if tag == 'ADJF':
    tag = tag[:-1]
if tag == 'ADVB':
    tag = tag[:-1]
if tag == 'ADJS':
    tag = tag[:-1]
if tag == 'NUMR':
    tag = tag[:-1]
if tag == 'PREP':
    tag = 'ADP'
if tag == 'CONJ':
    tag = 'CCONJ'
if tag == 'INFN':
    tag = 'VERB'
    
word += '_' + tag
print(word)
if word in model:
    res = model.most_similar(positive=[word], topn=7)
    for elems in res:
        print(elems[0])
else:
    print('в модели нет этого слова :(')

собачка
собачка_NOUN
собачонка_NOUN
собака_NOUN
болонка_NOUN
кошечка_NOUN
песик_NOUN
дворняжка_NOUN
пудель_NOUN


C:\Users\user\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


## Необязательное задание 2
(то же самое)

Напишите функцию, которая получает на вход предложение и заменяет часть слов на близкие по значению.

In [63]:
sent = input().split()
new_sent = []
print(sent)
for word in sent:
    w = pymorphy2_analyzer.parse(word)
    if w[0].tag.POS == 'ADJF':
        res = model.most_similar(positive=[w[0].normal_form + '_ADJ'], topn=1)
    else:
        res = model.most_similar(positive=[w[0].normal_form + '_' + w[0].tag.POS], topn=1)
    for elems in res:
        new_sent.append(elems[0])
print(new_sent)

мама мыла раму
['мама', 'мыла', 'раму']
['мама_PROPN', 'мыло_VERB', 'моторенваген_NOUN']


C:\Users\user\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [64]:
#вот это недоработанная версия


sent = input().split()
new_sent = []
print(sent)

def tagging(tag):
    if tag == 'ADJF':
        tag = tag[:-1]
    if tag == 'ADVB':
        tag = tag[:-1]
    if tag == 'ADJS':
        tag = tag[:-1]
    if tag == 'NUMR':
        tag = tag[:-1]
    if tag == 'PREP':
        tag = 'ADP'
    if tag == 'CONJ':
        tag = 'CCONJ'
    if tag == 'INFN':
        tag = 'VERB'
    return tag

for word in sent:
    w = pymorphy2_analyzer.parse(word)
    tag = word[0].tag.POS
    tagging(tag)
    res = model.most_similar(positive=[w[0].normal_form + '_' + tag], topn=1)
    for elems in res:
        new_sent.append(elems[0])
print(new_sent)
new1_sent = []
for elem in new_sent:
    new1_sent.append(elem[:-4])
print(new1_sent)

#вывести строчкой, убрать назв. части речи
#не работают местоимения, наречия, слово "черный", "родить", предлоги

мама мыла раму
['мама', 'мыла', 'раму']


AttributeError: 'str' object has no attribute 'tag'